In [3]:
!rm -fr date.ok
!ln -s ../1_getaro_datatime/date.ok

In [4]:
import netCDF4
import datetime
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import time as tt

In [5]:
time=datetime.datetime(2019, 1, 1 ,0,0)
time_end=datetime.datetime(2019, 1, 31 ,0,0)

url1='http://tds0.ifremer.fr/thredds/catalog/CORIOLIS-ARGO-GDAC-OBS/'
url3='/profiles/catalog.html'

data=pd.read_csv('date.ok', sep='\s+',header=None)
i=0
len(data)
for j in range(len(data)):
    url2=data[0][j][60:-17]
    url=url1+url2+url3
    res=r.get(url)
    soup=BeautifulSoup(res.text,'html.parser')


    d=[]
    for i in soup.select('tt'):
        d.append(i.text)

    a=[]
    t=[]
    for i in range(3,len(d),3):
        a.append(d[i])
        t.append(datetime.datetime(int(d[i+2][0:4]), int(d[i+2][5:7]), int(d[i+2][8:10]),0,0))


    for i in range(len(t)):
        if(t[i] >= time):
            url='http://tds0.ifremer.fr/thredds/dodsC/CORIOLIS-ARGO-GDAC-OBS/'+url2+'/profiles/'+str(a[i])
            tt.sleep(5)
            try:
                dataset = netCDF4.Dataset(url)
            except:
                try:
                    dataset = netCDF4.Dataset(url)
                except:
                    print(url)
            try:
                tin=datetime.datetime(1950, 1, 1,0,0) + datetime.timedelta(float(dataset['JULD'][0].data))
                if( (tin >=time) & (tin<=time_end) ) :
                    with open('data.final','a') as f:
                        f.writelines(url)
                        f.writelines('\n')
            except:
                pass
